# Python for Learning AI Week 2: TCP Client

Welcome to the client-side of our Port Communication exploration! This notebook focuses on client programming, teaching you how to connect to servers and exchange data. Understanding how clients work is essential for interacting with network services, APIs, and AI-powered applications.

## What You'll Learn
1. Creating TCP clients to connect to servers
2. Sending and receiving data over sockets
3. Making HTTP requests to web servers
4. Working with APIs (the client perspective)
5. Error handling in network communication

## Prerequisites
- Basic Python knowledge ([covered in Week 1](../week1/week1_python_basics.ipynb))
- Understanding of functions and error handling
- Companion notebook: 002_1_tcp_server.ipynb (server implementation)

Let's start by installing the required packages:

In [ ]:
# Check if required packages are installed
try:
    import socket  # For creating network connections
    import time    # For time-related functions
    import requests  # For making HTTP requests
    print("✓ All required packages are available!")
except ImportError as e:
    print(f"✗ Missing package: {str(e)}")
    print("\nIMPORTANT: To install packages, run this command in your terminal (not in the notebook):")
    print("pip install requests")
    print("\nWhy not install in the notebook?")
    print("- Installing via notebook (!pip install) only affects the current kernel")
    print("- Installing in the terminal ensures packages are available everywhere")
    print("- This ensures consistency across all notebooks and scripts")
    print("\nAfter installing, restart the kernel to use the new package.")

## Understanding Client-Server Communication

### Visual Client-Server Interaction

```
          CLIENT                             SERVER
        (Your App)                        (Remote Service)
            |                                   |
            |        (1) Connection Request     |
            |---------------------------------->|
            |                                   |
            |        (2) Connection Accepted    |
            |<----------------------------------|
            |                                   |
            |        (3) Send Request Data      |
            |---------------------------------->|
            |                                   |
            |        (4) Process Request        |
            |                                   |
            |        (5) Send Response Data     |
            |<----------------------------------|
            |                                   |
            |        (6) Close Connection       |
            |<---------------------------------→|
            |                                   |
```

### Client-Server Communication Ports

```
+-------------------+                      +------------------+
|                   |                      |                  |
|   CLIENT          |   Port 54321         |   SERVER         |
|   192.168.1.5     |--------------------->|   10.0.0.10      |
|                   |   connect to :8080   |   listening:8080 |
+-------------------+                      +------------------+
       |                                          |
       |                                          |
       v                                          v
   CLIENT USES                           SERVER LISTENS ON
   EPHEMERAL PORT                       WELL-KNOWN PORT
   (e.g., 54321-65535)                 (e.g., 80, 443, 8080)
```

Before we dive into the code, let's understand the client's role in network communication:

1. **Initiating Connection**: Unlike servers which wait passively, clients actively reach out to establish connections.

2. **Request-Response Pattern**: Clients typically send a request and wait for the server's response.

3. **Connection Management**: Clients need to properly open and close connections to avoid resource leaks.

4. **Error Handling**: Network operations can fail for various reasons, so clients must handle errors gracefully.

5. **Data Formatting**: Clients need to format outgoing data and parse incoming responses correctly.

In this notebook, we'll explore both low-level socket communication (TCP client) and high-level HTTP communication using the requests library.

## Creating a TCP Client

Let's create a simple client function that can connect to our TCP server from the companion notebook. 

Our client will:
- Establish a connection to the server
- Send a message
- Receive the response
- Properly close the connection

Here's the implementation:

In [ ]:
def run_tcp_client(host='localhost', port=12345, message="Hello, Server!"):
    """Run a simple TCP client that sends a message and receives a response"""
    # Create a socket object
    client_socket = None
    
    try:
        # Create the socket
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        print(f"Trying to connect to {host}:{port}...")
        # Connect to the server
        client_socket.connect((host, port))
        print(f"Connected to server at {host}:{port}")
        
        # Send a message to the server
        print(f"Sending message: {message}")
        client_socket.send(message.encode('utf-8'))
        
        # Receive the response
        response = client_socket.recv(1024).decode('utf-8')
        print(f"Received response: {response}")
        
        return response
        
    except ConnectionRefusedError:
        print("Connection refused. Make sure the server is running.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        # Always close the socket when done
        if client_socket:
            client_socket.close()
            print("Connection closed")

### Understanding the Client Function

Our `run_tcp_client` function follows these steps for network communication:

1. **Socket Creation**: 
   - Creates a new TCP socket for network communication
   - Uses the `socket.AF_INET` family (IPv4) and `socket.SOCK_STREAM` (TCP) type

2. **Connection Establishment**:
   - Uses the `connect()` method to reach the server at the specified host and port
   - Handles potential connection errors (like when the server isn't running)

3. **Data Exchange**:
   - Converts string messages to bytes with `encode('utf-8')` before sending
   - Receives response data and converts back to string with `decode('utf-8')`
   - Buffers up to 1024 bytes at a time

4. **Cleanup**:
   - Uses `finally` block to ensure the socket is always closed
   - Prevents resource leaks from lingering connections

This structure ensures resources are properly managed even if errors occur.

### Running the TCP Client

Now let's run our client to connect to the server. **Important: Before running this cell, make sure the TCP server is running in the companion notebook (002_1_tcp_server.ipynb).**

Try sending different messages to see how the server responds:

In [ ]:
# Run this after starting the server in the companion notebook
run_tcp_client(message="Hello from the client notebook!")

### Experiment: Changing Connection Parameters

Try experimenting with different connection parameters to better understand how TCP connections work:

1. **Change the message**:
   - Send a longer message
   - Send a message with special characters

2. **Change the port number**:
   - What happens when you use a different port than the server?
   - What happens when you use a well-known port (like 80) without privileges?

3. **Change the host**:
   - Try connecting to a non-existent host
   - Try connecting to a public address without a server

In [ ]:
# Experiment with different parameters
# For example, try a different port (server is on 12345)
# run_tcp_client(port=12346, message="Will this work?")

# Or try a different message
# run_tcp_client(message="こんにちは！")  # Hello in Japanese

## Creating an HTTP Client

While TCP sockets give us low-level control, most modern applications use higher-level protocols like HTTP. Python's `requests` library makes it easy to communicate with HTTP servers.

Let's create functions to interact with the HTTP server we set up in the companion notebook:

In [ ]:
def make_http_get_request(endpoint='', port=8000):
    """Make a GET request to the HTTP server"""
    url = f"http://localhost:{port}/{endpoint}"
    
    print(f"Making GET request to {url}")
    
    try:
        # Send GET request
        response = requests.get(url)
        
        # Check if successful
        response.raise_for_status()
        
        # Parse and return JSON
        data = response.json()
        print(f"Response: {data}")
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def make_http_post_request(endpoint='echo', data="Hello from HTTP client!", port=8000):
    """Make a POST request to the HTTP server"""
    url = f"http://localhost:{port}/{endpoint}"
    
    print(f"Making POST request to {url} with data: {data}")
    
    try:
        # Send POST request with data
        response = requests.post(url, data=data)
        
        # Check if successful
        response.raise_for_status()
        
        # Parse and return JSON
        result = response.json()
        print(f"Response: {result}")
        return result
        
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

### Testing the HTTP Client

Now let's test our HTTP client functions against the HTTP server from the companion notebook. 

**Important: Before running these cells, make sure the HTTP server is running in the server notebook (002_1_tcp_server.ipynb).**

First, let's try a GET request to different endpoints:

In [ ]:
# Test GET requests to different endpoints
# Root endpoint (/)
make_http_get_request(endpoint='')

# Time endpoint (/time)
# make_http_get_request(endpoint='time')

# Non-existent endpoint
# make_http_get_request(endpoint='does-not-exist')

Now, let's try a POST request to the echo endpoint:

In [ ]:
# Test POST request to the echo endpoint
make_http_post_request(endpoint='echo', data="This is a POST request from the client notebook!")

## Working with Real-World APIs

The techniques we've learned for HTTP clients are the same ones used to interact with real-world APIs, including AI services like OpenAI, Google's AI APIs, or your own AI services.

Here's an example structure of how you might use the same pattern to call an AI API (code is for illustration only):

```python
def call_ai_api(prompt, api_key):
    url = "https://api.example-ai.com/v1/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "prompt": prompt,
        "max_tokens": 100
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API call failed: {e}")
        return None
```

This shows how the client patterns you're learning are directly applicable to working with AI services.

## Practice Activities

Here are some activities to practice your client skills:

1. **Multiple Requests**: Modify the TCP client to send multiple messages in a single connection
2. **Timeout Handling**: Add a timeout to the client to prevent it from waiting indefinitely
3. **Custom Headers**: Add custom HTTP headers to your requests
4. **File Transfer**: Modify the TCP client to send the contents of a file to the server
5. **Error Recovery**: Implement retry logic when a connection fails

## Conclusion: Understanding Clients in Network Communication

In this notebook, we've covered:

1. **Creating TCP clients**: Using sockets to connect to TCP servers
2. **Data exchange**: Sending and receiving data over network connections
3. **HTTP client programming**: Using the requests library for HTTP communication
4. **API patterns**: Understanding how clients interact with APIs

These client-side skills are essential for working with AI services, as most state-of-the-art AI systems are accessed through API calls. Whether you're using OpenAI's GPT models, Hugging Face's APIs, or your own deployed models, you'll need to write client code to interact with them.

Remember, the client-server pattern is the foundation of modern distributed computing, and understanding both sides helps you build more robust and effective applications.